# PRTECAN Module Deep Dive Tutorial

This tutorial explores the `prtecan` module for processing Tecan plate reader data with a focus on:
- File parsing and data structures
- Titration curve analysis
- Advanced fitting capabilities
- Visualization techniques

In [ ]:
# Magic commands for development
%load_ext autoreload
%autoreload 2

# Setup
from pathlib import Path

import arviz as az
import matplotlib.pyplot as plt

from clophfit import prtecan
from clophfit.fitting.bayes import (
    fit_binding_pymc,
    fit_binding_pymc_compare,
)
from clophfit.fitting.core import (
    fit_binding_glob,
    fit_binding_glob_reweighted,
    outlier2,
)

# Configure notebook
%matplotlib inline
plt.style.use("seaborn-v0_8")


def output_fr(fr):
    print("Reduced X2:", fr.result.redchi)
    print("y_err_y1, y_err_y2:", (ds["y1"].y_err.mean(), ds["y2"].y_err.mean()))
    print(fr.dataset)
    return fr.figure


data_dir = Path("../../tests/Tecan/140220")

In [ ]:
from realistic_synthetic_data import generate_realistic_dataset, RealisticSimulationParameters

In [ ]:
import clophfit

In [ ]:
params = RealisticSimulationParameters(outlier_probability=0.8, outlier_magnitude=10)
ds = generate_realistic_dataset(params)
print(ds)
clophfit.fitting.core.fit_lm(ds, robust=1).figure

## 4. Fitting Framework

test:
- E10
- F10
- G09

TODO:

- simulate ds with
  - sigma1=100 sigma2=10
  - y1 with random outliers at pH < 6
  - extra buffer shift
  - heteroscedatic and omoscedastic error

- compare all functions
      

In [ ]:
# data_dir = Path("../../tests/Tecan/140220")
data_dir = Path("/home/dati/arslanbaeva/data/raw/L2/")

tit = prtecan.Titration.fromlistfile(data_dir / "list.pH.csv", is_ph=1)
tit.load_additions(data_dir / "additions.pH")
tit.load_scheme(data_dir / "scheme.txt")
tit.params.bg_mth = "meansd"
tit.bg_err

In [ ]:
tit.result_global.compute_all()

In [ ]:
tit.result_global.plot_k()

In [ ]:
from clophfit.testing.fitter_test_utils import make_synthetic_ds

In [ ]:
ds, truth = make_synthetic_ds(
    7, s1={"y1": 1000, "y2": 100}, s0={"y1": 800, "y2": 1200}, is_ph=1, noise=0.03
)

outlier2(ds, threshold=2.25).figure

In [ ]:
tit.bg_err

In [ ]:
k = "A01"

ds = tit._create_global_ds(k)
ds

In [ ]:
output_fr(fit_binding_glob(ds))

In [ ]:
fr = outlier2(ds, k, plot_z_scores=1, threshold=3.0)
output_fr(fr)

In [ ]:
fr.result.logger = 1

In [ ]:
ds2 = tit._create_ds(k, 2)
ds2

In [ ]:
k = "A01"
fr = tit.result_global[k]
fr.figure

In [ ]:
fr.dataset

In [ ]:
{"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()}

In [ ]:
n_sd = 0.15 / fr.result.params["K"].stderr
print(n_sd)

# Run the model with a single noise scaling factor
trace_single = fit_binding_pymc_compare(
    fr,
    {"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()},
    n_sd=max(n_sd, 1),
    n_xerr=0.682,
    learn_separate_y_mag=False,
)

# Run the model with separate noise scaling factors for each label
trace_separate = fit_binding_pymc_compare(
    fr,
    {"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()},
    n_sd=max(n_sd, 1),
    n_xerr=0.682,
    learn_separate_y_mag=True,
)
trace_separate_shot = fit_binding_pymc_compare(
    fr,
    {"y1": tit.bg_err[1].mean(), "y2": tit.bg_err[2].mean()},
    n_sd=max(n_sd, 1),
    n_xerr=0.682,
    learn_separate_y_mag=True,
)

In [ ]:
az.summary(trace_separate_shot)

In [ ]:
az.summary(trace_single)

In [ ]:
az.summary(trace_separate)

In [ ]:
n_sd = 0.15 / fr.result.params["K"].stderr
print(n_sd)
fr_mcmc2 = fit_binding_pymc(fr, n_sd=max(n_sd, 1), n_xerr=0.682)

In [ ]:
output_fr(fr_mcmc2)
df = az.summary(fr_mcmc2.mini)
df

In [ ]:
az.plot_trace(fr_mcmc2.mini)

In [ ]:
obs_ll_vars = [f"y_likelihood_{lbl}" for lbl in ds]  # ds is your dict-like Dataset
loo = az.loo(fr_mcmc2, var_name=obs_ll_vars)

In [ ]:
az.loo(fr_mcmc2.mini, var_name=["y_likelihood_y1", "y_likelihood_y2"])

In [ ]:
fr_mcmc2.dataset["y1"]

In [ ]:
# da1.mask = ~outlier_glob(fr.result.residual, plot_z_scores=1, threshold=2)

In [ ]:
output_fr(fit_binding_glob_reweighted(ds, k, threshold=2.25))

In [ ]:
from clophfit.fitting import core

In [ ]:
output_fr(core.fit_lm(ds))

In [ ]:
output_fr(core.fit_lm(ds, robust=True))

In [ ]:
output_fr(core.fit_lm(ds, iterative=True))

In [ ]:
output_fr(core.fit_lm(ds, outlier_threshold=1.9))